# MTHexapod Following Error

In [ ]:
# Times Square parameters
day_obs = 20250420
sal_index = 2

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML
from pathlib import Path

from lsst.summit.utils.efdUtils import makeEfdClient

import Hexapod_Following_Error as hex_debug

In [ ]:
# Initialize an EFD client
efd_client = makeEfdClient()

# Global variables
HEXAPOD_AXES = ["X", "Y", "Z", "U", "V", "W"]
N_AXES = len(HEXAPOD_AXES)
N_STRUTS = 6

## Data Analysis

In [ ]:
log_messages = hex_debug.query_hexapod_log_messages(efd_client, day_obs, sal_index)
controller_states = hex_debug.query_hexapod_controller_state(efd_client, day_obs, sal_index)

hex_debug.collapsible_table(efd_client, log_messages, controller_states, "message", sal_index, expanded=False)